In [34]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.datasets import load_iris
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
import numpy as np
from sklearn import linear_model
import scipy.io as spio
from sklearn.multioutput import MultiOutputRegressor
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.gaussian_process import GaussianProcessRegressor, kernels
import shelve

In [86]:
lattice = 'cubic'
coeff=0
data = spio.loadmat(lattice+'-data-posd-with-den.mat')
X = data['xdata']
y = data['ydata'][:,coeff]

In [3]:
models_and_parameters = {
  'lasso': (linear_model.Lasso(),
              {'reg__alpha': [0.01, 0.1, 0.5, 1.,5.,10.]}),
    'elnet': (linear_model.ElasticNet(),
              {'reg__alpha':[0.01, 0.1, 0.5, 1, 5., 10.], 'reg__l1_ratio':[0.,0.1,0.5,1.,2.1]}),
    'krg': (KernelRidge(),
            {'reg__kernel':['rbf','linear'], 'reg__alpha': [1e0, 0.1, 1e-2, 1e-3], 'reg__gamma': np.logspace(-2, 2, 5)}),
    'gpr': (GaussianProcessRegressor(kernel = kernels.RBF()),
            {'reg__kernel__length_scale':[0.01, 0.1, 1., 2., 10., 100.], 'reg__kernel__length_scale_bounds':[(1e-2,1.),(1e-1,1.),(1e-1,10.),(1.,10.),(1.,100.)\
,(1e-2,1e2)]}),
    'gbr': (GradientBoostingRegressor(learning_rate=0.01, min_samples_split=2, max_features='sqrt', loss='ls', subsample=0.4),
            {'reg__max_depth': [2,3,4,10,20,50],'reg__min_samples_leaf': [2,3,4,10], 'reg__learning_rate':[0.01, 0.1], 'reg__max_features':['auto', 'sqrt', 'l\
og2']}),
    'ada': (AdaBoostRegressor(base_estimator=DecisionTreeRegressor(),n_estimators=500,learning_rate=0.01),#max_depth alone doesn't work probably               
            {'reg__base_estimator__max_depth': [2,3,4,10], 'reg__base_estimator':[DecisionTreeRegressor(max_depth = 4, max_features='auto'),
                                                                                     DecisionTreeRegressor(max_depth = None, max_features='auto'),
                                                                                     DecisionTreeRegressor(max_depth = 4, max_features='sqrt'),
                                                                                     DecisionTreeRegressor(max_depth = None, max_features='sqrt')]}),
    'svr': (SVR(),
            {'reg__C': [0.01, 0.05, 0.1, 1], 'reg__kernel': ['linear', 'rbf']}),
    'rf': (RandomForestRegressor(),
           {'reg__max_depth': [None, 5, 10, 50]}),
    'brg': (linear_model.BayesianRidge(fit_intercept=True),
            {'reg__alpha_1': [1.e-6, 1.e-5]}),
    'lars': (linear_model.Lars(fit_intercept = True, normalize=False),
             {'reg__n_nonzero_coefs': [5, 10, 50, 500, np.inf]}),
    'ard': (linear_model.ARDRegression(),
            {'reg__alpha_1':[1.e-6, 1.e-5]})}

In [37]:
scaler = preprocessing.StandardScaler()
inner_cv = KFold(n_splits=3, shuffle=True)
outer_cv = KFold(n_splits=3, shuffle=True)

In [6]:
model=RandomForestRegressor(n_estimators=1000)
params={'reg__max_depth': [None, 5, 10, 50,100,200],'reg__max_features':['auto','sqrt','log2'],'reg__min_samples_split':[2,3,4],'reg__min_samples_leaf':[2,3,4]}
pipeline = Pipeline([('transformer', scaler), ('reg', model)])

In [9]:
clf = GridSearchCV(estimator=pipeline, param_grid=params, cv=inner_cv,n_jobs=-1)
clf

GridSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=True),
       error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('transformer', StandardScaler(copy=True, with_mean=True, with_std=True)), ('reg', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurit...ators=1000, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'reg__max_depth': [None, 5, 10, 50, 100, 200], 'reg__max_features': ['auto', 'sqrt', 'log2'], 'reg__min_samples_split': [2, 3, 4], 'reg__min_samples_leaf': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [39]:
model=RandomForestRegressor(n_estimators=1000,
                            max_depth=200,
                            max_features='sqrt',
                            min_samples_leaf= 2,
                            min_samples_split= 2)
pipeline = Pipeline([('transformer', scaler), ('reg', model)])
reg1 = Pipeline([('transformer', scaler), ('reg', model)])
#clf.fit(X,y)
#clf.best_params_

In [47]:
reg1.fit(X,y)
scores=cross_val_score(reg1, X=X, y=y, scoring='neg_mean_squared_error',cv=outer_cv)
r2scores=cross_val_score(reg1, X=X, y=y, scoring='r2',cv=outer_cv)
rmse_scores = [np.sqrt(abs(s)) for s in scores]
print('Cross-validation results:')
print('Folds: %i, mean RMSE: %.3f' % (len(scores), np.mean(np.abs(rmse_scores))))
print('Folds: %i, mean r2: %.3f' % (len(r2scores), np.mean(r2scores)))

Cross-validation results:
Folds: 3, mean RMSE: 45.679
Folds: 3, mean r2: 0.646


In [48]:
X.shape

(170, 20)

In [49]:
ntdata = spio.loadmat(lattice+'-non-training-data-with-den.mat')
Xnt = ntdata['xntdata']
ynt1=reg1.predict(Xnt)

In [11]:
#Old part
clf.fit(X,y)
scores=cross_val_score(clf, X=X, y=y, scoring='neg_mean_squared_error',cv=outer_cv)
r2scores=cross_val_score(clf, X=X, y=y, scoring='r2',cv=outer_cv)
rmse_scores = [np.sqrt(abs(s)) for s in scores]
print('Cross-validation results:')
print('Folds: %i, mean RMSE: %.3f' % (len(scores), np.mean(np.abs(rmse_scores))))
print('Folds: %i, mean r2: %.3f' % (len(r2scores), np.mean(r2scores)))

Cross-validation results:
Folds: 3, mean RMSE: 48.200
Folds: 3, mean r2: 0.587


In [50]:
from matminer.figrecipes.plotly.make_plots import PlotlyFig

pf_rf = PlotlyFig(x_title='DFT (MP) C11 (GPa)',
                  y_title='Random forest C11 (GPa)',
                  plot_title='Random forest regression',
                  plot_mode='offline',
                  margin_left=150,
                  textsize=35,
                  ticksize=30,
                  filename="rf_regression.html")

# a line to represent a perfect model with 1:1 prediction
xy_line = {'x_col': [min(y), max(y)],
           'y_col': [min(y), max(y)],
           'color': 'black',
           'mode': 'lines',
           'legend': None,
           'text': None,
           'size': None}


pf_rf.xy_plot(x_col=y,
              y_col=reg1.predict(X),
              size=3,
              marker_outline_width=0.5,
              #text=df_mp['pretty_formula'],
              add_xy_plot=[xy_line])

In [56]:
coeff=1
y = data['ydata'][:,coeff]
params=models_and_parameters['brg'][1]
scaler = preprocessing.StandardScaler()
inner_cv = KFold(n_splits=3, shuffle=True)
outer_cv = KFold(n_splits=3, shuffle=True)

In [59]:
model=linear_model.BayesianRidge(normalize=False)
scaler = preprocessing.StandardScaler()
pipeline = Pipeline([('transformer', scaler), ('reg', model)])
reg2 = GridSearchCV(estimator=pipeline, param_grid=params, cv=inner_cv,n_jobs=-1)
reg2

GridSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=True),
       error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('transformer', StandardScaler(copy=True, with_mean=True, with_std=True)), ('reg', BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
       fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
       normalize=False, tol=0.001, verbose=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'reg__alpha_1': [1e-06, 1e-05]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [60]:
reg2.fit(X,y)
reg2.best_params_
scores=cross_val_score(reg2, X=X, y=y, scoring='neg_mean_squared_error',cv=outer_cv)
r2scores=cross_val_score(reg2, X=X, y=y, scoring='r2',cv=outer_cv)
rmse_scores = [np.sqrt(abs(s)) for s in scores]
print('Cross-validation results:')
print('Folds: %i, mean RMSE: %.3f' % (len(scores), np.mean(np.abs(rmse_scores))))
print('Folds: %i, mean r2: %.3f' % (len(r2scores), np.mean(r2scores)))

Cross-validation results:
Folds: 3, mean RMSE: 16.895
Folds: 3, mean r2: 0.812


In [61]:
from matminer.figrecipes.plotly.make_plots import PlotlyFig
pf_rf = PlotlyFig(x_title='DFT (MP) C12 (GPa)',
                  y_title='Bayesian Ridge C12 (GPa)',
                  plot_title='Random forest regression',
                  plot_mode='offline',
                  margin_left=150,
                  textsize=35,
                  ticksize=30,
                  filename="br2_regression.html")

# a line to represent a perfect model with 1:1 prediction
xy_line = {'x_col': [min(y), max(y)],
           'y_col': [min(y), max(y)],
           'color': 'black',
           'mode': 'lines',
           'legend': None,
           'text': None,
           'size': None}


pf_rf.xy_plot(x_col=y,
              y_col=reg2.predict(X),
              size=3,
              marker_outline_width=0.5,
              #text=df_mp['pretty_formula'],
              add_xy_plot=[xy_line])

In [62]:
coeff=2
y = data['ydata'][:,coeff]
params=models_and_parameters['brg'][1]
scaler = preprocessing.StandardScaler()
inner_cv = KFold(n_splits=3, shuffle=True)
outer_cv = KFold(n_splits=3, shuffle=True)
model=linear_model.BayesianRidge(normalize=False)
scaler = preprocessing.StandardScaler()
pipeline = Pipeline([('transformer', scaler), ('reg', model)])


In [64]:
reg3 = GridSearchCV(estimator=pipeline, param_grid=params, cv=inner_cv,n_jobs=-1)
reg3
reg3.fit(X,y)
reg3.best_params_
scores=cross_val_score(reg3, X=X, y=y, scoring='neg_mean_squared_error',cv=outer_cv)
r2scores=cross_val_score(reg3, X=X, y=y, scoring='r2',cv=outer_cv)
rmse_scores = [np.sqrt(abs(s)) for s in scores]
print('Cross-validation results:')
print('Folds: %i, mean RMSE: %.3f' % (len(scores), np.mean(np.abs(rmse_scores))))
print('Folds: %i, mean r2: %.3f' % (len(r2scores), np.mean(r2scores)))

Cross-validation results:
Folds: 3, mean RMSE: 15.297
Folds: 3, mean r2: 0.677


In [65]:
from matminer.figrecipes.plotly.make_plots import PlotlyFig
pf_rf = PlotlyFig(x_title='DFT (MP) C44 (GPa)',
                  y_title='Bayesian Ridge C44 (GPa)',
                  plot_title='Random forest regression',
                  plot_mode='offline',
                  margin_left=150,
                  textsize=35,
                  ticksize=30,
                  filename="br3_regression.html")

# a line to represent a perfect model with 1:1 prediction
xy_line = {'x_col': [min(y), max(y)],
           'y_col': [min(y), max(y)],
           'color': 'black',
           'mode': 'lines',
           'legend': None,
           'text': None,
           'size': None}

pf_rf.xy_plot(x_col=y,
              y_col=reg3.predict(X),
              size=3,
              marker_outline_width=0.5,
              #text=df_mp['pretty_formula'],
              add_xy_plot=[xy_line])

In [113]:
y1t=reg1.predict(X)
y2t=reg2.predict(X)
y3t=reg3.predict(X)
e1t=(y1t+2*y2t)/3
e2t=(y1t-y2t)
e3t=y3t
from sklearn.metrics import *
yall = data['ydata']
print(mean_squared_error(y1t,yall[:,0]))
print(mean_squared_error(y2t,yall[:,1]))
print(mean_squared_error(y3t,yall[:,2]))
print(mean_squared_error(e1t,(yall[:,0]+2.*yall[:,1])/3.))
print(mean_squared_error(e2t,yall[:,0]-yall[:,1]))
print(mean_squared_error(e3t,yall[:,2]))
print()
print(r2_score(y1t,yall[:,0]))
print(r2_score(y2t,yall[:,1]))
print(r2_score(y3t,yall[:,2]))
print(r2_score(e1t,(yall[:,0]+2.*yall[:,1])/3.))
print(r2_score(e2t,yall[:,0]-yall[:,1]))
print(r2_score(e3t,yall[:,2]))

490.3637129223002
192.77643830499622
153.3538356875204
189.15275066377413
462.68750631145537
153.3538356875204

0.8699361771486456
0.8575116845181366
0.7003761886091838
0.8876006525944001
0.7839085382760193
0.7003761886091838


In [121]:
y1nt=reg1.predict(Xnt)
y2nt=reg2.predict(Xnt)
y3nt=reg3.predict(Xnt)

In [133]:
ynt=np.column_stack((y1nt,y2nt,y3nt))
ynt.shape
spio.savemat(lattice+'-nt-result',mdict={'coeffsnt':ynt,'coord':ntdata['coord'],'mps':ntdata['mps'],'volrat':ntdata['volrat']})

In [132]:
ntdata

{'__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Thu Feb 15 19:16:02 2018',
 '__version__': '1.0',
 'coord': array([[10.16054111, 10.16049212,  9.16737424, 10.16045947, 10.16055907,
         10.16060969, 10.16060969, 10.16060962, 10.1605803 , 10.16053621,
         11.99978583, 10.16054111, 11.99994707, 11.99994707, 11.99994707,
         10.16060969, 10.160595  , 11.99994707, 10.16052152, 12.        ,
         10.1602235 , 10.16052968, 10.16045947, 11.99994707, 10.16049212,
         11.99994707, 11.99880118, 10.16053621,  9.74762969,  7.99999999,
         10.16060965, 10.16060967, 10.16060969, 10.16060969, 10.06017725,
          9.36586487,  8.75173199,  8.84809999,  9.17562831,  9.28155295,
         10.16060968,  8.98079857,  5.99999978, 10.16060964, 10.16049212,
          5.62991395, 11.99999999, 10.16060969, 10.16055962, 10.16049212,
         10.16060968,  9.51932521, 10.16055091,  4.84345909, 10.16050682,
          6.02654411, 10.16049212, 10.160